In [1]:
#imports
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid

In [2]:
#Parse Input
raw_text = ""
pdf_reader = PdfReader("Input/input.pdf")
for page in pdf_reader.pages:
    raw_text += page.extract_text()

#Split data
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
chunks = splitter.split_text(raw_text)
print(len(chunks))


20


In [3]:
#Build input Dataframe
rows = []
for chunk in chunks:
    row = {
        "text": chunk,
        "chunk_id": uuid.uuid4().hex,
    }
    rows = rows + [row]
df = pd.DataFrame(rows)
print(df)

                                                 text  \
0   Review began\n 05/30/2023 \nReview ended\n 06/...   
1   and the burgeoning private health sector syste...   
2   materials, extracting the data, and analyzing ...   
3   who does not have a formal medical qualificati...   
4   [12]\n. ANMs are government employees and rece...   
5   payments are delayed, and the temporary nature...   
6   more available and of a higher quality. This t...   
7   “star” doctors with flourishing practices can ...   
8   healthcare they can ill afford. The Government...   
9   locations on the premise that a set percentage...   
10  2023 Saxena et al. Cureus 15(6): e40274. DOI 1...   
11  smoking, alcohol abuse, physical inactivity, a...   
12  and staff shortages \n[52]\n. Sourcing health ...   
13  and the recommendations made here are an effor...   
14  . Accessed: October 12, 2021:\nhttps://data.wo...   
15  As good as physicians: patient perceptions of\...   
16  Reforming the regulation of

In [5]:
#RUNNING THIS COSTS MONEY!!!!!!!!!!!!!!!
#----------------------------------------------------------
#Get the concepts
from graph_helper import get_concepts
from graph_helper import formatDF

generate_dataframes = False
if(generate_dataframes):
    concepts = df.apply(lambda row: get_concepts(row), axis=1)
    concepts = concepts.dropna()
    concepts = concepts.reset_index(drop=True)
    concept_list = np.concatenate(concepts).ravel().tolist()
    dfg1 = formatDF(concept_list)

    dfg1.to_csv("Output/graph.csv", sep="|", index=False)
    df.to_csv("Output/chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv("output/graph.csv", sep="|")
dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
 
print(dfg1.shape)
print(dfg1.head())

(310, 4)
  node_1                                             node_2  \
0  india                                  health indicators   
1  india                                         population   
2  india                             health workers density   
3  india                                      who threshold   
4  india  skewed inter-state, urban-rural, and public-pr...   

                                                edge  count  
0  India's health indicators have improved in rec...      4  
1       The country with a population of 1.3 billion      4  
2  has an estimated active health workers density...      4  
3  which is much lower than the WHO threshold of ...      4  
4  The issue is compounded by the skewed inter-st...      4  


In [7]:
dfg_long = pd.melt(
        df, value_vars=["node_1", "node_2"], value_name="node"
    )
print(dfg_long)

KeyError: "The following 'value_vars' are not present in the DataFrame: ['node_1', 'node_2']"